In [4]:
import os
import boto3
import logging
from datetime import datetime
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

In [5]:
import sagemaker
from sagemaker.transformer import Transformer

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [6]:
sgmkr_clnt = boto3.client("sagemaker")
sgmkr_rntm = boto3.client("runtime.sagemaker")

In [7]:
role_arn = sagemaker.get_execution_role()
model_img = "811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest"
model_path = ("s3://sagemakerlearningwithiris/iris/model/iris-xgboost2024-09-25-19-15-18/output/model.tar.gz")
model_name = "iris-xgboost" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(model_name)

iris-xgboost2024-09-27-13-08-17


In [8]:
response = sgmkr_clnt.create_model(
    ModelName = model_name,
    PrimaryContainer = {"Image":model_img, "ModelDataUrl":model_path},
    ExecutionRoleArn = role_arn
)
print(response)

{'ModelArn': 'arn:aws:sagemaker:us-east-1:216393269487:model/iris-xgboost2024-09-27-13-08-17', 'ResponseMetadata': {'RequestId': 'd49419e2-42c0-49dd-9886-6fd47638b6ab', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd49419e2-42c0-49dd-9886-6fd47638b6ab', 'content-type': 'application/x-amz-json-1.1', 'content-length': '93', 'date': 'Fri, 27 Sep 2024 13:08:17 GMT'}, 'RetryAttempts': 0}}


In [9]:
bucket = "sagemakerlearningwithiris"
prefix = "iris"

### Batch Transform

In [10]:
batch_ip = "s3://{}/{}/{}".format(bucket, prefix, "batch_transform")
batch_op = "s3://{}/{}/{}".format(bucket, prefix, "batch_transform")

In [11]:
transformer = Transformer(
    model_name=model_name,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path=batch_op
)

In [12]:
# transformer.transform(data=batch_ip, data_type="S3Prefix", content_type="text/csv")

In [15]:
from datetime import datetime 
sgmkr_clnt = boto3.client("sagemaker")
sgmkr_rntime = boto3.client("runtime.sagemaker")
ep_config_name = "tmp-ep-config" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
response = sgmkr_clnt.create_endpoint_config(
    EndpointConfigName=ep_config_name,
    ProductionVariants=[
        {
            "VariantName":"version-1",
            "ModelName":model_name,
            "InitialInstanceCount":1,
            "InstanceType": "ml.m4.xlarge",
        }
    ]
)

In [16]:
print(response)

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:216393269487:endpoint-config/tmp-ep-config2024-09-27-13-10-35', 'ResponseMetadata': {'RequestId': 'e0a4e8b3-7c52-4944-97af-1c4949a03933', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e0a4e8b3-7c52-4944-97af-1c4949a03933', 'content-type': 'application/x-amz-json-1.1', 'content-length': '113', 'date': 'Fri, 27 Sep 2024 13:10:35 GMT'}, 'RetryAttempts': 0}}


In [19]:
ep_name = "tmp-end" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S-%f")
print(ep_name)

tmp-end2024-09-27-13-14-35-432808


In [23]:
response = sgmkr_clnt.create_endpoint(
    EndpointName=ep_name, 
    EndpointConfigName=ep_config_name
    )
waiter = sgmkr_clnt.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=ep_name, WaiterConfig={"Delay": 123, "MaxAttempts": 123})
print("Endpoint created")

ClientError: An error occurred (ValidationException) when calling the CreateEndpoint operation: Cannot create already existing endpoint "arn:aws:sagemaker:us-east-1:216393269487:endpoint/tmp-end2024-09-27-13-14-35-432808".

### Error Above occurs due trying to create an endpoint that already exist

In [26]:
payload = "7.7, 3.0, 6.1, 2.3"
response = sgmkr_rntm.invoke_endpoint(EndpointName=ep_name, ContentType="text/csv", Body=payload)
print(response["Body"].read().decode())

2.0


In [28]:
print("Deleting sagemaker endpoint")
response = sgmkr_clnt.delete_endpoint(EndpointName = ep_name)  
print("Deleted sagemaker endpoint")
#print(response)

Deleting sagemaker endpoint
Deleted sagemaker endpoint


In [30]:
sgmkr_clnt.delete_endpoint_config(EndpointConfigName=ep_config_name)

{'ResponseMetadata': {'RequestId': '4d92c7ea-27f8-4862-a35b-d91fb21ad6bc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4d92c7ea-27f8-4862-a35b-d91fb21ad6bc',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 27 Sep 2024 13:24:26 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}